In [1]:
import numpy as np 
import random, operator, math
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
class Projection:

    def __init__(self, type, c):

        assert c > 0

        if type == 'exponential':
            assert c <= .01
            def equation(x):  return c * math.pow(x,2)
        elif type == 'logistic': 
            assert c <= 50
            def equation(x): return c * math.log10(x)
        elif type == 'linear':
            assert c <= 1
            def equation(x): return c * x
        else: raise Exception('Invalid type')

        self.equation = equation

exponential = Projection('exponential', .01)
assert exponential.equation(1) == .01
assert round(exponential.equation(50), 0) == 25
assert exponential.equation(100) == 100

logistic = Projection('logistic', 10)
assert logistic.equation(1) == 0
assert round(logistic.equation(50), 0) == 17
assert logistic.equation(100) == 20

linear = Projection('linear', .5)
assert linear.equation(1) == .5
assert linear.equation(50) == 25
assert linear.equation(100) == 50

In [3]:
class Asset:

    def __init__(self, id, risk, principal, projection):

        self.id = id
        self.risk = risk
        self.principal = principal
        self.projection = projection

    def valuation(self, current, target, risk_tol):
        subjective_risk = pow(self.risk, 1 + risk_tol)
        total_risk = subjective_risk * (target - current) / 100
        return self.principal * self.projection.equation(target) * (1 - total_risk)

penny = Asset(id = 0,
              risk = .8,
              principal = 100,
              projection = exponential)

stock = Asset(id = 1,
              risk = .5,
              principal = 100,
              projection = linear)

bond = Asset(id = 2,
             risk = .2,
             principal = 100,
             projection = logistic)

print('true penny projections ( ', penny.principal, 'principal )')
print({1: penny.projection.equation(1) * penny.principal,
       25: penny.projection.equation(25) * penny.principal,
       100: penny.projection.equation(100) * penny.principal})

print('penny projections for low risk')
print({1: penny.valuation(0, 1, .1),
       25: penny.valuation(15, 25, .1),
       100: penny.valuation(75, 100, .1)})

print('penny projections for mid risk')
print({1: penny.valuation(0, 1, .5),
       25: penny.valuation(15, 25, .5),
       100: penny.valuation(75, 100, .5)})

print('penny projections for high risk')
print({1: penny.valuation(0, 1, .9),
       25: penny.valuation(15, 25, .9),
       100: penny.valuation(75, 100, .9)})

print('--------------------------------------')

print('true stock projections (', stock.principal, 'principal )')
print({1: stock.projection.equation(1) * stock.principal,
       25: stock.projection.equation(25) * stock.principal,
       100: stock.projection.equation(100) * stock.principal})

print('stock projections for low risk')
print({1: stock.valuation(0, 1, .1),
       25: stock.valuation(15, 25, .1),
       100: stock.valuation(75, 100, .1)})

print('stock projections for mid risk')
print({1: stock.valuation(0, 1, .5),
       25: stock.valuation(15, 25, .5),
       100: stock.valuation(75, 100, .5)})

print('stock projections for high risk')
print({1: stock.valuation(0, 1, .9),
       25: stock.valuation(15, 25, .9),
       100: stock.valuation(75, 100, .9)})

print('--------------------------------------')

print('true bond projections (', bond.principal, 'principal )')
print({1: bond.projection.equation(1) * bond.principal,
       25: bond.projection.equation(25) * bond.principal,
       100: bond.projection.equation(100) * bond.principal})

print('bond projections for low risk')
print({1: bond.valuation(0, 1, .1),
       25: bond.valuation(15, 25, .1),
       100: bond.valuation(75, 100, .1)})

print('bond projections for mid risk')
print({1: bond.valuation(0, 1, .5),
       25: bond.valuation(15, 25, .5),
       100: bond.valuation(75, 100, .5)})

print('bond projections for high risk')
print({1: bond.valuation(0, 1, .9),
       25: bond.valuation(15, 25, .9),
       100: bond.valuation(75, 100, .9)})


true penny projections (  100 principal )
{1: 1.0, 25: 625.0, 100: 10000.0}
penny projections for low risk
{1: 0.9921765378516566, 25: 576.1033615728536, 100: 8044.134462914143}
penny projections for mid risk
{1: 0.9928445824720007, 25: 580.2786404500042, 100: 8211.145618000168}
penny projections for high risk
{1: 0.9934555828315932, 25: 584.097392697457, 100: 8363.895707898284}
--------------------------------------
true stock projections ( 100 principal )
{1: 50.0, 25: 1250.0, 100: 5000.0}
stock projections for low risk
{1: 49.7667417521158, 25: 1191.6854380289496, 100: 4416.854380289496}
stock projections for mid risk
{1: 49.82322330470336, 25: 1205.8058261758408, 100: 4558.058261758408}
stock projections for high risk
{1: 49.866028317182966, 25: 1216.5070792957408, 100: 4665.070792957408}
--------------------------------------
true bond projections ( 100 principal )
{1: 0.0, 25: 1397.9400086720377, 100: 2000.0}
bond projections for low risk
{1: 0.0, 25: 1374.1375658986067, 100: 191

In [13]:
class Agent:

    def __init__(self, risk_tol, goal, horizon, money):

        self.risk_tol = risk_tol
        self.goal = goal
        self.horizon = horizon
        self.assets = {}
        self.money = money
    
    def add_asset(self, asset):

        self.assets[asset.id] = asset

    def remove_asset(self, id):

        self.assets.pop(id)

    def buy(self, asset, price):

        self.assets[asset.id] = asset
        self.money = self.money - price

    def sell(self, asset, price):

        self.assets.pop(asset.id)
        self.money = self.money + price

    def get_info(self, asset):

        self.assets[asset.id] = asset
    
    def evaluate(self, id, current, target):
        return self.assets[id].valuation(current, target, self.risk_tol)

a = Agent(risk_tol = .5,
          goal = 1000,
          horizon = 80,
          money = 200)

a.add_asset(stock)

b = Agent(risk_tol = .5,
          goal = 500,
          horizon = 50,
          money = 200)

b.add_asset(bond)

print(a.evaluate(1, 0, 20))
print(b.evaluate(2, 0, 20))
        

929.2893218813452
929.2893218813452
1277.756463575414
